# EOF Analysis of AR days

* Multivariate EOF analysis in T-mode
* K-means clustering

In [2]:
# Import Python modules
import os, sys
from pathlib import Path
import numpy as np
import numpy.ma as ma
import pandas as  pd
import xarray as xr
from sklearn.cluster import KMeans
# matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.colors import ListedColormap
from matplotlib import rcParams
# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

# Path to modules
sys.path.append('../modules')

# Import my modules
from plotter import draw_basemap
from timeseries import persistence
from eofs import *
from ar_funcs import ar_climatology
from kmeans import *

In [ ]:
# Set up paths

home = Path.home()                                # users home directory
root = home/'DATA'/'repositories'/'AR_types'      # project root directory
path_to_data = home/'DATA'/'data'                 # project data -- read only
path_to_out  = root/'out'                         # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = root/'figs'                        # figures

# check that path exists
path_to_figs.exists()

In [ ]:
# Set a default font for all matplotlib text (can only set this ONCE; must restart kernel to change it)

rcParams['font.family'] = 'sans-serif'   # set the default font family to 'sans-serif'
rcParams['font.sans-serif'] = 'Arial'    # set the default sans-serif font to 'Arial'

## Data

### AR time series

In [ ]:
# Read CSV data into pandas DataFrame
filename = path_to_data / 'CH1_generated_data/ar_catalog_fraction_HASIAsubregions.nc'
ds = xr.open_dataset(filename)

#For MERRA2 IVT
ds = ds.sel(time=slice('1980-01-01', '2018-12-31'))

R01_dates = ar_climatology(ds.R01, 0.3)
R02_dates = ar_climatology(ds.R02, 0.3)
R03_dates = ar_climatology(ds.R03, 0.3)


In [ ]:
thres = 0.3

df = ds.to_dataframe()
## drop lev and ens cols
df = df.drop(columns=['lev', 'ens'])
## resample to daily
df = df.resample('1D').mean()
# Add column of AR days based on threshold
# (no LLJ day eq 0; LLJ day eq 1)
df['ar'] = 0
idx = (df['R01'] > thres) | (df['R02'] > thres) | (df['R03'] > thres)
df.loc[idx, 'ar'] = 1

# Add column of AR locations 
# ('R01', 'R02', 'R03', 'R01/R02', 'R02/R03', 'R01/R03', 'R01/R02/R03', nan)
df['location'] = np.nan

idx = (df['R01'] >= thres) & (df['R02'] < thres) & (df['R03'] < thres)
df.loc[idx, 'location'] = 'R01'

idx = (df['R01'] < thres) & (df['R02'] >= thres) & (df['R03'] < thres)
df.loc[idx, 'location'] = 'R02'

idx = (df['R01'] < thres) & (df['R02'] < thres) & (df['R03'] >= thres)
df.loc[idx, 'location'] = 'R03'

idx = (df['R01'] >= thres) & (df['R02'] >= thres) & (df['R03'] < thres)
df.loc[idx, 'location'] = 'R01/R02'

idx = (df['R01'] < thres) & (df['R02'] >= thres) & (df['R03'] >= thres)
df.loc[idx, 'location'] = 'R02/R03'

idx = (df['R01'] >= thres) & (df['R02'] < thres) & (df['R03'] >= thres)
df.loc[idx, 'location'] = 'R01/R03'

idx = (df['R01'] >= thres) & (df['R02'] >= thres) & (df['R03'] >= thres)
df.loc[idx, 'location'] = 'R01/R02/R03'

# Show table
df.head()

### MERRA2 reanalysis

In [ ]:
# Select lat/lon grid
lonmin = 0
lonmax = 120
latmin = 0
latmax =  50

### MERRA2 DATA ###
def preprocess(ds):
    '''keep only selected lats and lons'''
    return ds.sel(lat=slice(latmin, latmax), lon=slice(lonmin, lonmax))

filepath_pattern = '/home/sbarc/students/nash/data/MERRA2/anomalies/IVT/daily_*.nc'
merra = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, combine='by_coords')
print('ds size in GB {:0.2f}\n'.format(merra.nbytes / 1e9))

# ## Use if IVT
# new_times = pd.date_range('1980-01-01-09', '2018-12-31-09', freq='D')
# ds_z['time'] = new_times
# ds_z['ivt'] = np.sqrt(ds_z.ivtx**2 + ds_z.ivty**2)
merra
# ds_z = ds_z.load()

In [ ]:
# Read datafiles into xarray datasets
# f2 = xr.open_dataset(path_to_data/'era5.sam.05dg.ivtn.1979-2016.nc')

# Add LLJ time series to era5; set as coordinate variables
merra['ar'] = ('time', df.ar)
merra = merra.set_coords('ar')

merra['location'] = ('time', df.location)
merra = merra.set_coords('location')

# print dataset
print(merra)

### Data Subset Selection

In [ ]:
# Trim date range
start_date = '1980-12-01'
end_date = '2018-02-28'
idx = slice(start_date, end_date)
merra = merra.sel(time=idx)

# Select NDJFM months
idx = (merra.time.dt.month >= 12) | (merra.time.dt.month <= 2)
merra = merra.sel(time=idx)

# Select AR days
idx = (merra.ar >= 1)
merra_ar = merra.sel(time=idx)

# print results
print(merra_ar)

In [ ]:
# Count number of independent AR events

years = np.arange(1980, 2018) 
nyrs = len(years)
total_events = 0
for k in range(nyrs-1):    
    # Extract single DJF season
    date1 = "{}-12-01".format(years[k])
    date2 = "{}-02-28".format(years[k+1])
    x = merra.ar.sel(time=slice(date1,date2)).values
    # Count AR events in that season
    tags, tmp = persistence(x)
    # Add to running event count
    total_events += tmp

print("Number of independent AR events: ", total_events)

### Climatology and Anomalies

In [ ]:
# # Mean IVT of AR days in DJF
# merra_ar_clim = merra_ar.mean(dim='time')
# #print(merra_ar_clim, '\n')

# # IVT Anomalies
# merra_ar['ivtx'] = era_llj.ivte - era_llj_clim.ivte
# era_llj['ivtn_anom'] = era_llj.ivtn - era_llj_clim.ivtn
# #print(era_llj)

## Preprocessing

### Reshape, center, and standardize data matrix

In [ ]:
%%time
# Load merra_ar dataset into memory
merra_ar = merra_ar.load()


In [ ]:
def center_data(in_array):
    ''' Remove the mean of an array along the first dimension.
    
    If *True*, the mean along the first axis of *dataset* (the
    time-mean) will be removed prior to analysis. If *False*,
    the mean along the first axis will not be removed. Defaults
    to *True* (mean is removed).
    The covariance interpretation relies on the input data being
    anomaly data with a time-mean of 0. Therefore this option
    should usually be set to *True*. Setting this option to
    *True* has the useful side effect of propagating missing
    values along the time dimension, ensuring that a solution
    can be found even if missing values occur in different
    locations at different times.
    '''
    # Compute the mean along the first dimension.
    mean = in_array.mean(axis=0, skipna=False)
    # Return the input array with its mean along the first dimension
    # removed.
    return (in_array - mean)

def remove_missing_values(X):
    # Find the indices of values that are missing in the whole row
    nonMissingIndex = np.where(np.logical_not(np.isnan(X[0])))[0]
    # Remove missing values from the design matrix.
    dataNoMissing = X[:, nonMissingIndex]
    return nonMissingIndex, dataNoMissing

def valid_nan(in_array):
    inan = np.isnan(in_array)
    return (inan.any(axis=0) == inan.all(axis=0)).all()

def standardize_and_flatten_arrays(arr1, arr2):

    # Data dimensions with missing values removed
    ntim, npts = arr1.shape

    # Transpose arrays to get [space x time]
    X1 = arr1.T
    X2 = arr2.T

    # Standardize by columns
    x1std = np.std(X1, axis=0)
    X1s = X1 / x1std

    x2std = np.std(X2, axis=0)
    X2s = X2 / x2std

    # Combine variables into single data matrix Xs
    Xs = np.empty((nvar*npts,ntim))
    Xs[0:npts,:] = X1s
    Xs[npts:,:]  = X2s
    print(Xs.shape)

    # Check that column means=0 and std dev=1
    test = np.mean(np.mean(Xs, axis=0))
    print("Column means: ", np.round(test,2))
    test = np.mean(np.std(Xs, axis=0))
    print("Column std: ", np.round(test,2))
    
    return Xs

In [ ]:
# Center the variables by removing long-term mean
var1 = center_data(merra_ar.ivtx)
var2 = center_data(merra_ar.ivty)

# Weight the data by the square root of the cosine of the lat
wgts = spatial_weights(merra_ar.lat)
var1 = var1*wgts
var2 = var2*wgts

In [ ]:
%%time
# Extract variables as numpy arrays
var1 = var1.values
var2 = var2.values

# Data dimensions
ntim, nlat, nlon = var1.shape
npts = nlat*nlon
nvar = 2

# Reshape into 2D arrays by flattening the spatial dimension
tmp1 = np.reshape(var1, (ntim, npts))
tmp2 = np.reshape(var2, (ntim, npts))

# Remove missing data
tmp1_idx, tmp1_miss = remove_missing_values(tmp1)
tmp2_idx, tmp2_miss = remove_missing_values(tmp2)

## Test if the removal of nans was successful
print(valid_nan(tmp1_miss), valid_nan(tmp2_miss))

## Standardize and flatten variable arrays
Xs = standardize_and_flatten_arrays(tmp1_miss, tmp2_miss)
Xs_nomiss = standardize_and_flatten_arrays(tmp1, tmp2)

## EOF Analysis

In [ ]:
%%time
# Compute eigenvalues & eigenvectors
evals, evecs = calc_eofs(Xs)

print('Eigenvalues: ', evals.shape)
print(evals, '\n')

print('Eigenvectors: ', evecs.shape)
print(np.round(evecs, 3), '\n')

### Explained Variance

In [ ]:
def pct_variance(eig):
    var_eig = eig/sum(eig)*100
    return var_eig

In [ ]:
# Calculate the percent explained var by each eigenvector
pctvar = pct_variance(evals)

# Number of EOFs that explain more than 1% of the total variance
idx = pctvar[pctvar >= 1.0]
neofs = len(idx)

# print exp var >= 1.0
cumvar = np.sum(pctvar[0:neofs-1])
print(f'Cumulative variance explained by the first {neofs} EOFs:')
print(f'{cumvar:.2f}% \n')

# print exp var: neofs = 4
cumvar = np.sum(pctvar[0:3])
print(f'Cumulative variance explained by the first 4 EOFs:')
print(f'{cumvar:.2f}% \n')

# print exp var for 4 eofs
for k in range(4):
    print(f'{k+1} \t {pctvar[k]:.2f}%')

### North Test

In [ ]:
err = north_test(evals, total_events)
upper = pctvar + err
lower = pctvar - err

print(np.round(upper[0:6],3))
print(np.round(pctvar[0:6],3))
print(np.round(lower[0:6],3))

### Fig 2: Variance

In [ ]:
# set seaborn style
sns.set()
sns.set_style("ticks", {'patch.force_edgecolor':False})

# create figure
fig, ax = plt.subplots(figsize=(6,4))

# plot data
xvals = np.arange(neofs) + 1
ax.bar(xvals, pctvar[0:neofs], yerr=err[0:neofs], 
       color='tab:blue', alpha=0.8)

# x-axis
ax.set_xlabel('EOF')
ax.set_xticks(xvals)

# y-axis
ax.set_ylabel('Explained Variance (%)')
yticks = np.arange(0,16,3)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks) 

# save fig
filepath = path_to_figs / 'fig2.png'
plt.savefig(filepath, dpi=300)

# show
plt.show()

### Loadings

In [ ]:
neofs = 19
loads = loadings(evals, evecs, neofs)

print(loads.shape)
print(np.round(loads,3))

### Save EOFs

In [ ]:
# Save eigenvalues, eigenvectors, and loadings

neofs = 4   # number of EOFs to save (evecs, loadings3)

outfile = path_to_out / 'eigenvalues.txt'
np.savetxt(outfile, evals, fmt='%.5f')

outfile = path_to_out / 'eigenvectors.txt'
np.savetxt(outfile, evecs[:,0:neofs], fmt='%.5f', delimiter=',')

outfile = path_to_out / 'loadings.txt'
np.savetxt(outfile, loads[:,0:neofs], fmt='%.4f', delimiter=',')


### PCs

In [ ]:
# Calculate principal components (spatial modes)
neofs = 19
pcs = calc_pcs(Xs_nomiss, evecs, neofs)
# pcs = calc_pcs(Xs, evecs, neofs)

In [ ]:
# npts = len(tmp1_idx)

# # Combine missing idx into single data matrix
# tmp_idx = np.empty((nvar*npts))
# print(tmp_idx.shape)
# tmp_idx[0:npts] = tmp1_idx
# tmp_idx[npts:]  = tmp2_idx
# print(tmp_idx.shape)

# ## create an array of nans that is the size of the original unmasked array
# pcs_comb = np.ones([neofs, nvar*nlat*nlon], dtype=Xs.dtype) * np.NaN
# ## fill in array with values of pcs at indexed values from tmp_idx
# for count, ele in enumerate(tmp_idx):
#     pcs_comb[:, int(ele)] = pcs[:,count]
# print(pcs_comb.shape)

In [ ]:
# Split pcs into separate arrays for each variable
tmp1 = pcs[:,0:npts]
tmp2 = pcs[:,npts:]
# tmp1 = pcs_comb[:,0:nlat*nlon]
# tmp2 = pcs_comb[:,nlat*nlon:]

# Reshape spatial dim back to 2D map
pcmodes_var1 = np.reshape(tmp1, (neofs,nlat,nlon))
pcmodes_var2 = np.reshape(tmp2, (neofs,nlat,nlon))
#print(pcmodes_var1.shape, pcmodes_var2.shape)

### Fig 3: Spatial Modes

In [ ]:
# Panel Plot of Spatial Modes

# number of eofs to plot
neofs = 4

# Data for plotting
lons = merra_ar.lon.data
lats = merra_ar.lat.data
udat = pcmodes_var1[0:neofs,:,:]
vdat = pcmodes_var2[0:neofs,:,:]
# data = np.sqrt(udat**2 + vdat**2)
data = udat**2
print(np.nanmin(data), np.nanmax(data))

# Set up projection
mapcrs = ccrs.PlateCarree()
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
dx = np.arange(lonmin,lonmax+20,20)
dy = np.arange(latmin,latmax+20,20)

# subtitles
eof_label = [ ]
var_label = [ ]
for k in range(neofs):
    eof_label.append("EOF{:1d}".format(k+1,))
    var_label.append("{:.2f}%".format(pctvar[k]))

In [ ]:
# Create figure
fig = plt.figure(figsize=(10,11))
nrows = 2
ncols = 2

sns.set_style('ticks')

# Set up Axes Grid
axes_class = (GeoAxes,dict(map_projection=mapcrs))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(nrows, ncols), axes_pad = 0.55,
                cbar_location='bottom', cbar_mode='single',
                cbar_pad=0.0, cbar_size='2.5%',label_mode='')

#newcmap = cmocean.tools.crop_by_percent(cmo.matter, 15, which='max', N=None)

# Loop for drawing each plot
for k, ax in enumerate(axgr):
    ax = draw_basemap(ax, extent=[lonmin,lonmax,latmin,latmax], xticks=dx, yticks=dy)
    
    # Add contour fill plot
    clevs = np.arange(0,100,5)
    cf = ax.contourf(lons, lats, data[k,:,:], transform=datacrs,
                     levels=clevs,
                     cmap="Blues", extend='max')
    # add vectors
    ax.quiver(lons, lats, udat[k,:,:], vdat[k,:,:], transform=datacrs,
              color='black', pivot='middle', regrid_shape=20)      
    # subtitles
    ax.set_title(eof_label[k], loc='left', fontsize=12)
    ax.set_title(var_label[k], loc='right', fontsize=12)
    
# single colorbar
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
cb.set_label('kg m$^{-1}$ s$^{-1}$', fontsize=11)
cb.ax.tick_params(labelsize=10)
    
# Display figure
filepath = path_to_figs / 'eofs.png'
plt.savefig(filepath, dpi=200, bbox_inches='tight')
plt.show()

## K means clustering

In [3]:
# Determine optimal K

# maximum number of clusters (number of iterations)
kmax =15
# number of eofs
neofs = 4
# input data
xdata = loads[:,0:neofs]

# Elbow plot
outfile = path_to_figs / 'xfig1.png'
plot_optimal_k(xdata, kmax, filename=outfile)


NameError: name 'loads' is not defined

In [ ]:
## K-means cluster analysis

# Number of clusters
nk = 4

# Input data
xdata = loads[:,0:neofs]

# Compute k means and assign each point to a cluster
kmeans = KMeans(n_clusters=nk)
kmeans.fit(xdata)
cluster = kmeans.predict(xdata)

# LLJ category labels (llj days only)
llj_cat = cluster + 1


In [ ]:
# Count number of days in each cluster
klabels, counts = np.unique(llj_cat, return_counts=True)

# Save counts to txt file
res = np.column_stack((klabels,counts))
headstr = 'LLJ_TYPE, COUNT'
outfile = path_to_out / 'k_counts.txt'
np.savetxt(outfile, res, delimiter=',', fmt='%d', header=headstr)


In [ ]:
# Cluster centroids (nclust x neofs)
centroids = kmeans.cluster_centers_

# Save centroids to txt file
res = np.column_stack((klabels,centroids))
headstr = "LLJ_TYPE, EOF1, EOF2, EOF3, EOF4"
outfile = path_to_out / 'centroids.txt'
np.savetxt(outfile, res, delimiter=',', fmt='%s', header=headstr)


### Save LLJ category labels

In [ ]:
## Save LLJ location, loadings (EOF1-4), and category label (LLJ days only)

# Vector of LLJ dates
dates_lljDays = era_llj.time.values

# Create new dataframe
data = {'LOC':era_llj.location.values,
        'EOF1':loads[:,0],
        'EOF2':loads[:,1],
        'EOF3':loads[:,2],
        'EOF4':loads[:,3],
        'LLJ_CAT':llj_cat}
df_out = pd.DataFrame(data, index=dates_lljDays)
print(df_out)

# Export dataframe as csv
outfile = path_to_out / 'sallj-types-loadings.csv'
df_out.to_csv(outfile)


In [ ]:
## Save time series of all NDJFM days with SALLJ types

# Arrays with ALL NDJFM days
dates_allDays = era.time.values
llj_cat_allDays = np.zeros(len(dates_allDays), dtype=int)

# Loop over llj days and match to llj_full 
for i, date in enumerate(dates_lljDays):
    idx = np.where(dates_allDays == date)
    llj_cat_allDays[idx] = llj_cat[i]  

# Create dataframe
data = {'LLJ_CAT':llj_cat_allDays}
df_out = pd.DataFrame(data, index=dates_allDays)
print(df_out)

outfile = path_to_out / 'sallj-types-ndjfm.csv'
df_out.to_csv(outfile)
